# Problem

Real estate is changing. Deep learning is leading the way to many new methods of determing how to price a property at a competitive rate. Traditionally, when a seller is ready to put a property on the market, they would contact an agent to start the process. In recent times, this process has changed; we now have iBuyers in the market that can harness the details (They ask sellers fill out a short form online or accept a call to answer questions about your home features, updates and condition, photos to make you the seller a compepetitive cash offer).  
Real estate is complex. Multiple of levels of complexity e.g transactions, additions, updates. Zingat operates as an intermediary between potential buyers and sellers in the real estate market. In this regard, to help both parties they routinely value and monitor real estate market. For this purpose, They regularly build new and better models using new technologies in the field of data science. Not only with automated valuation models that predict house prices, they also employ time series models to understand market trends. 
Zingat has made available a public dataset on kaggle (https://www.kaggle.com/zingatbi/zingat-real-estate) to build our version of a basic Automated Valuation Model. We will use modern methods to predict the price of a property using core principles of Comparable Market Analysis (our basic assumption is that a competitve offer price is a price at which similar or comparable properties in the same area recently sold).
Based on this assumption we will introduce the idea of merging multiple statistical models to output an offer amount for a property. 
We will test these model restricted set of future data to glean the effectiveness of our generalizations. This side project is inspired by the wonderful white paper from Kristoffer Birkeland and Allan Daniel D'Silva from the Norwegian Univeristy of Science and Technology.
The dataset we are using in this project covers Turkey. However, the accuracy of these ensemble models will be extended to other locations via service API to benchmark it against real estate brokers and the Zestimate. 


# Domain History 

The use of data science in econometrics has enormous potential. Traditional methods used historically to predict in this domain 
include K-means, Least Squares, complex regressions e.t.c 
Some more modern approaches include ensemble learning (finite set of alternative models) and Artifical Neural Networks (Kaggle is a great resource for this). 
These methods having popualrized by big time iBuyers like Zillow which publishes the awesome Zestimate (https://www.zillow.com/how-much-is-my-home-worth)
The value of an AVM in real estate is a positive since it can be useful for anyone involved in the buisness (useful for fair competition). 
As mentioned a great solution (take for example the Zestimate) should be flexible, multi-input (home prices, seller provided details, external details e.t.c).


# Preprocessing Data

We will begin by mapping out our data by location (focusing on address)


In [1]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
print(os.listdir('../Zingat-Property-Automated-Valuation-Model'))

['.git', '.ipynb_checkpoints', 'AVM.ipynb', 'README.md', 'real_estate_data.csv']


In [61]:
# reading csv file from url  


data = pd.read_csv('real_estate_data.csv', dtype=object)

In [76]:
data

,id,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,address,furnished,heating_type,price,price_currency
0,1,Konut,Rezidans,12/10/18,1/9/19,2,30,0,20 ve üzeri,2,2+1,90,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,3500,TRY
1,2,Konut,Daire,2/13/19,NaN,1,14,0,20 ve üzeri,20 ve üzeri,1+0,43,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,490000,TRY
2,3,Konut,Daire,10/9/18,11/8/18,1,30,0,1,Yüksek Giriş,2+1,NaN,Tekirdağ/Çorlu/Reşadiye,NaN,Fancoil,155000,TRY
3,4,Konut,Rezidans,9/10/18,10/10/18,1,30,3,20 ve üzeri,20 ve üzeri,6+1,450,İstanbul/Beşiktaş/Levent,NaN,Fancoil,32500000,TRY
4,5,Konut,Rezidans,12/10/18,1/9/19,1,30,0,20 ve üzeri,2,2+1,90,İstanbul/Kartal/Kordonboyu,NaN,Fancoil,1450000,TRY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
403482,403483,Konut,Daire,9/18/18,NaN,2,162,NaN,NaN,NaN,+,NaN,İstanbul/Sultanbeyli/Adil,NaN,NaN,1500,TRY
403483,403484,Konut,Daire,10/11/18,NaN,1,139,NaN,NaN,NaN,2+1,NaN,Sakarya/Adapazarı/Cumhuriyet,NaN,NaN,120000,TRY
403484,403485,Konut,Daire,11/22/18,NaN,1,97,NaN,NaN,NaN,1+1,NaN,Antalya/Alanya/Saray,NaN,NaN,48000,EUR
403485,403486,Konut,Daire,2/21/19,NaN,2,6,NaN,NaN,NaN,2+1,2,Aydın/Kuşadası/Türkmen,NaN,NaN,900,TRY


In [90]:
df = data.set_index('size')
df[['First','Last','f']] = df['address'].str.split("/",expand=True)
df.join(df2)

ValueError: Columns must be same length as key

In [97]:
data[['First','Last','f']] = data.address.str.split(('/',2).tolist(),expand=True,)

AttributeError: 'tuple' object has no attribute 'tolist'

In [83]:
data

,id,type,sub_type,start_date,end_date,listing_type,tom,building_age,total_floor_count,floor_no,room_count,size,address,furnished,heating_type,price,price_currency,First,Last,third


In [78]:
# dropping null value columns to avoid errors 
data.dropna(inplace = True) 

In [74]:
# new data frame with split value columns  
df = pd.DataFrame(data.address.str.split('/',2).tolist(),
                                 columns = ['city','county','district'])

In [75]:
df

,city,county,district
0,İstanbul,Kartal,Kordonboyu
1,İstanbul,Kartal,Kordonboyu
2,Tekirdağ,Çorlu,Reşadiye
3,İstanbul,Beşiktaş,Levent
4,İstanbul,Kartal,Kordonboyu
...,...,...,...
403482,İstanbul,Sultanbeyli,Adil
403483,Sakarya,Adapazarı,Cumhuriyet
403484,Antalya,Alanya,Saray
403485,Aydın,Kuşadası,Türkmen


In [37]:
# making separate first name column from new data frame 
data["city"]= new[0] 

KeyError: 0

In [34]:

  
# dropping null value columns to avoid errors 
data.dropna(inplace = True) 
  
# new data frame with split value columns 
new = data["address"].str.split("/", n = 1, expand = True) 
  
# making separate first name column from new data frame 
data["city"]= new[0] 
  
# making separate last name column from new data frame 
data["county"]= new[1] 

# making separate last name column from new data frame 
data["district"]= new[2]

# Dropping old Name columns 
data.drop(columns =["address"], inplace = True) 
  
# df display 
data 

KeyError: 0